<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/Llama3_2_Multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from huggingface_hub import login

# Replace 'your-hf-token-here' with your actual Hugging Face token
login(token="xxx")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
!huggingface-cli download meta-llama/Llama-3.2-3B --include "original/*" --local-dir Llama-3.2-3B

Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading 'original/consolidated.00.pth' to 'Llama-3.2-3B/.cache/huggingface/download/original/consolidated.00.pth.e85237a45033593adb3eb7e1a6cc5c410c4e2b360e24a422d3c1ab12a166c441.incomplete'

original/params.json: 100% 220/220 [00:00<00:00, 1.30MB/s]
Download complete. Moving file to Llama-3.2-3B/original/params.json

consolidated.00.pth:   0% 0.00/6.43G [00:00<?, ?B/s]

tokenizer.model:   0% 0.00/2.18M [00:00<?, ?B/s]
consolidated.00.pth:   0% 10.5M/6.43G [00:00<01:15, 85.3MB/s]
consolidated.00.pth:   0% 31.5M/6.43G [00:00<00:42, 149MB/s] 
consolidated.00.pth:   1% 62.9M/6.43G [00:00<00:28, 219MB/s]

tokenizer.model: 100% 2.18M/2.18M [00:00<00:00, 5.83MB/s]
Download complete. Moving file to Llama-3.2-3B/original/tokenizer.model

consolidated.00.pth:   1% 94.4M/6.43G [00:00<00:27, 228MB/s]
consolidated.00.pth:   2% 126M/6.43G [00:00<00:25, 243MB/s] 
consolidated.00.pth:   2% 157M/6.43G [00:00<00:28, 217MB/s]
consolidated.00.pth:   3% 189M/6

In [6]:
!huggingface-cli download meta-llama/Llama-3.2-1B --include "original/*" --local-dir Llama-3.2-1B

Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading 'original/consolidated.00.pth' to 'Llama-3.2-1B/.cache/huggingface/download/original/consolidated.00.pth.cfa47fb7185ebd1c3712fff5fa04fdbb0675c1e345fc72e12f6c7b65b8da6f05.incomplete'

consolidated.00.pth:   0% 0.00/2.47G [00:00<?, ?B/s]

tokenizer.model:   0% 0.00/2.18M [00:00<?, ?B/s]Downloading 'original/params.json' to 'Llama-3.2-1B/.cache/huggingface/download/original/params.json.e1ea21c4ff6e1905e1f42ecf8ebe0ee10a063b98.incomplete'
tokenizer.model: 100% 2.18M/2.18M [00:00<00:00, 26.8MB/s]
Download complete. Moving file to Llama-3.2-1B/original/tokenizer.model

consolidated.00.pth:   0% 10.5M/2.47G [00:00<00:46, 52.9MB/s]

original/params.json: 100% 220/220 [00:00<00:00, 991kB/s]
Download complete. Moving file to Llama-3.2-1B/original/params.json

consolidated.00.pth:   1% 31.5M/2.47G [00:00<00:24, 101MB/s] 
consolidated.00.pth:   3% 62.9M/2.47G [00:00<00:16, 149MB/s]
consolidated.00.pth:   4% 94.4M/2.47G [00:00<00:13, 180MB/s]
c

In [7]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

pipe("The key to life is")

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'The key to life is the ability to make a choice. It is the ability to make a'}]

In [9]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

pipe("The key to life is")

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'The key to life is the ability to accept and appreciate the life you have.\nA man once'}]

In [11]:
!pip -q install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.6 MB/s eta 0:00:00


#Restart the session please!

In [1]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

In [3]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

prompt = "<|image|><|begin_of_text|>If I had to write a haiku for this one"
inputs = processor(image, prompt, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=30)
print(processor.decode(output[0]))


config.json:   0%|          | 0.00/5.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

<|begin_of_text|><|image|><|begin_of_text|>If I had to write a haiku for this one, it would be:.\nPeter Rabbit is a rabbit.\nHe lives in a hole in the ground.\nHe's a bit of a
